In [576]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler


DATA_ROOT = 'D:\\100days\data\ml100marathon-02-01'

In [577]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dftest = dftest[~dftest.Coupon_id.isnull()]
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)
dfoff.head(20)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN


In [578]:
dfoff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160742 entries, 0 to 1160741
Data columns (total 7 columns):
User_id          1160742 non-null int64
Merchant_id      1160742 non-null int64
Coupon_id        746969 non-null float64
Discount_rate    746969 non-null object
Distance         1090916 non-null float64
Date_received    746969 non-null float64
Date             456709 non-null float64
dtypes: float64(4), int64(2), object(1)
memory usage: 62.0+ MB


In [579]:
##plt.bar(dfoff['Date_received'],dfoff['User_id'])
#dfoff = sns.load_dataset("dfoff")
#ax = sns.countplot(x="User_id", hue="Date_received", data=dfoff)

#import seaborn as sns
#sns.set(style="darkgrid")
#dfoff = sns.load_dataset("dfoff") #titanic经典数据集，带有登船人员的信息
#源数据集class代表三等舱位，who代表人员分类，男女小孩，对每一类人数计数
#ax = sns.countplot(hue="User_id", x="Date_received", data=dfoff)

In [580]:
print('有优惠券，购买商品數目', dfoff[~np.isnan(dfoff['Coupon_id']) & ~np.isnan(dfoff['Date'] )].shape[0])
print('无优惠券，购买商品數目', dfoff[np.isnan(dfoff['Coupon_id']) & ~np.isnan(dfoff['Date'])].shape[0])
print('有优惠券，没有购买商品數目', dfoff[~np.isnan(dfoff['Coupon_id'] ) & np.isnan(dfoff['Date'] )].shape[0])
print('无优惠券，没有购买商品數目', dfoff[np.isnan(dfoff['Coupon_id']) & np.isnan(dfoff['Date'])].shape[0])
print('-----------------------------------------------------------------------------------------------------------------')
print('有優惠券，購買商品條數', dfoff[(pd.notnull(dfoff['Date_received'])) & (pd.notnull(dfoff['Date']))].shape[0])
print('無優惠券，購買商品條數',  dfoff[(pd.isnull(dfoff['Date_received'])) & (pd.notnull(dfoff['Date']))].shape[0])
print('有優惠券，不購買商品條數',  dfoff[(pd.notnull(dfoff['Date_received'])) & (pd.isnull(dfoff['Date']))].shape[0])
print('無優惠券，不購買商品條數', dfoff[(pd.isnull(dfoff['Date_received'])) & (pd.isnull(dfoff['Date']))].shape[0])


有优惠券，购买商品數目 42936
无优惠券，购买商品數目 413773
有优惠券，没有购买商品數目 704033
无优惠券，没有购买商品數目 0
-----------------------------------------------------------------------------------------------------------------
有優惠券，購買商品條數 42936
無優惠券，購買商品條數 413773
有優惠券，不購買商品條數 704033
無優惠券，不購買商品條數 0


In [581]:
# 在測試集中出現的用戶但訓練集沒有出現
print('1. User_id in training set but not in test set', set(dftest['User_id']) - set(dfoff['User_id']))
# 在測試集中出現的商戶但訓練集沒有出現
print('2. Merchant_id in training set but not in test set', set(dftest['Merchant_id']) - set(dfoff['Merchant_id']))

1. User_id in training set but not in test set {6553604, 1835014, 786439, 2883597, 2097168, 7340050, 7340060, 1048607, 1835039, 6553633, 2883618, 2097187, 4194340, 5505061, 5505064, 5505068, 1572909, 1572911, 3670069, 7077943, 3932215, 3932220, 2883646, 1572937, 2621513, 1310793, 262221, 262222, 3145809, 3407956, 7340117, 786520, 4194399, 3670111, 2359392, 3670114, 5242982, 1572967, 1835112, 3407977, 6029421, 524412, 5243009, 4456581, 524426, 6291595, 3670157, 2883725, 786574, 5505167, 4456594, 5767327, 6029471, 166, 262314, 4718763, 2883757, 5243057, 4718775, 6291639, 5243065, 1573047, 3932352, 4456642, 3145926, 262343, 6815944, 4980941, 1048782, 5243089, 7340245, 5767382, 4456662, 5243096, 1310935, 5505241, 6553825, 7078117, 1573095, 3408104, 6029545, 3408106, 6815979, 1835243, 239, 3145971, 1310968, 5767418, 786682, 1048832, 5767427, 1310987, 5505292, 6553877, 7078173, 1573150, 1311010, 1835299, 7078182, 1573166, 1048879, 6291761, 3670321, 262452, 3146041, 1573178, 2097468, 3146044,


2. Merchant_id in training set but not in test set {2, 2053, 2055, 4106, 11, 8203, 12, 8205, 2061, 6155, 8219, 2078, 32, 8227, 36, 2084, 2087, 4135, 2090, 43, 2091, 8237, 4139, 4145, 2098, 8244, 8249, 4157, 62, 8255, 2117, 70, 71, 4165, 73, 8266, 4170, 6214, 6219, 8271, 82, 84, 85, 8281, 6235, 4188, 94, 8287, 4196, 102, 107, 108, 6252, 8302, 6260, 6262, 2171, 127, 4223, 4226, 2183, 8328, 4232, 8331, 4237, 142, 2190, 4240, 148, 8342, 6301, 6305, 6307, 6312, 6314, 6315, 2237, 190, 4285, 6335, 193, 6338, 8392, 2248, 2250, 4299, 8397, 2254, 4302, 6350, 210, 4306, 2260, 6356, 4310, 4311, 2264, 6364, 6365, 223, 2271, 8418, 226, 4322, 8423, 8424, 2281, 234, 235, 6377, 2286, 8431, 4334, 4336, 2290, 6382, 8439, 248, 6395, 6398, 4354, 4357, 8457, 4361, 2315, 268, 4363, 2321, 274, 6418, 8468, 277, 6425, 6427, 4381, 4382, 6429, 8480, 6431, 4387, 4388, 8486, 2344, 6442, 2349, 2351, 4401, 6450, 311, 6457, 8507, 4412, 318, 6463, 2370, 323, 327, 8520, 6473, 336, 4436, 2392, 2394, 347, 6493, 4448, 854

In [582]:
print('Discount_rate 类型:',dfoff['Discount_rate'].unique())
print('Distance 类型:', dfoff['Distance'].unique())

Discount_rate 类型: [nan '20:1' '200:20' '10:5' '100:10' '200:30' '20:5' '30:5' '50:10'
 '150:10' '100:30' '200:50' '100:50' '300:30' '50:20' '0.9' '50:5' '150:20'
 '10:1' '30:10' '30:1' '0.95' '100:5' '50:1' '100:20' '0.8' '300:20'
 '100:1' '20:10' '0.85' '0.6' '5:1' '150:30' '200:10' '300:50' '0.5'
 '200:5' '0.7' '0.75' '30:20' '0.2' '150:50' '300:10' '50:30' '150:5'
 '200:100']
Distance 类型: [  0.   1.   2.  nan  10.   4.   7.   9.   3.   5.   6.   8.]


In [583]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [584]:
print(dfoff.head(20))

    User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0   1439408         2632        NaN           NaN       0.0            NaN   
1   1439408         2632     8591.0          20:1       0.0     20160217.0   
2   1439408         2632     1078.0          20:1       0.0     20160319.0   
3   1832624         3381     7610.0        200:20       0.0     20160429.0   
4   2029232         3381    11951.0        200:20       1.0     20160129.0   
5   2223968         3381     9776.0          10:5       2.0     20160129.0   
6     73611         2099    12034.0        100:10       NaN     20160207.0   
7    163606         1569     5054.0        200:30      10.0     20160421.0   
8   3273056         4833     7802.0        200:20      10.0     20160130.0   
9     94107         3381     7610.0        200:20       2.0     20160412.0   
10   253750         8390        NaN           NaN       0.0            NaN   
11   253750         8390     7531.0          20:5       0.0     

In [585]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x : 1 if x >=5 else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].apply(lambda x : 1 if x >=5 else 0 ) # apply to testset


dfoff.head(40)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,1
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,1
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,1
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,1
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN,0,7.0,1
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0,6.0,1
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0,2.0,0


In [586]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

dfoff.head(20)

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,0,0,0,0,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,1,0,0,0,0,0,1,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,1,0,0,0,0,1,0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,1,0,0,0,0,1,0,0
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,1,0,0,0,0,1,0,0
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN,0,7.0,1,0,0,0,0,0,0,1
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,0,0,0,1,0,0,0
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0,6.0,1,0,0,0,0,0,1,0
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0,2.0,0,0,1,0,0,0,0,0


In [587]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def date_convet(Date):
    if 'nan' in str(Date):
        return np.nan
    else:
        datestr = str(int(Date))
        return pd.to_datetime(datestr) 


   
    
def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
 
    # convert distance
    df.loc[df.Distance.isnull(), "Distance"] = 99
    return df

    # convert distance
   # df['distance'] = df['Distance'].replace('null', -1).astype(int)
   # print(df['distance'].unique())
   # return df

dfoff = processData(dfoff)
dftest = processData(dftest)

dfoff.head(20)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,...,0,0,0,0,0,0,NaN,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,0,1,0,0,0,0,0.950000,20,1,1
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,1,...,0,0,0,0,1,0,0.950000,20,1,1
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,1,...,0,0,0,1,0,0,0.900000,200,20,1
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,1,...,0,0,0,1,0,0,0.900000,200,20,1
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,1,...,0,0,0,1,0,0,0.500000,10,5,1
6,73611,2099,12034.0,100:10,99.0,20160207.0,NaN,0,7.0,1,...,0,0,0,0,0,1,0.900000,100,10,1
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,...,0,0,1,0,0,0,0.850000,200,30,1
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0,6.0,1,...,0,0,0,0,1,0,0.900000,200,20,1
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0,2.0,0,...,1,0,0,0,0,0,0.900000,200,20,1


In [588]:
# convert distance
dfoff['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dfoff['distance'].unique())
dftest['distance'] = dftest['Distance'].replace('null', -1).astype(int)
print(dftest['distance'].unique())

[ 0  1  2 99 10  4  7  9  3  5  6  8]
[ 1  0 99  7  2 10  9  4  8  3  6  5]


In [589]:
# convert distance
dfoff['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dfoff['distance'].unique())
dftest['distance'] = dftest['Distance'].replace('null', -1).astype(int)
print(dftest['distance'].unique())

[ 0  1  2 99 10  4  7  9  3  5  6  8]
[ 1  0 99  7  2 10  9  4  8  3  6  5]


In [597]:
#特征工程

#对时间类特征进行简单处理
#将优惠券领取的月份，星期几，和一年中的星期几
df.loc[:,'date_received_new'] = df['Date_received'].apply(date_convet)
df.loc[:,'date_received_month'] = df['date_received_new'].dt.month
df.loc[:,'date_received_dom'] = df['date_received_new'].dt.day
df.loc[:,'date_received_doy'] = df['date_received_new'].dt.dayofyear
df.loc[:,'date_received_dow'] = df['date_received_new'].dt.dayofweek

In [599]:
# 对日期型数据进行one-hot编码
month_onehot = pd.get_dummies(df[['date_received_month']].astype(str))
dom_onehot = pd.get_dummies(df[['date_received_dom']].astype(str))
dow_onehot = pd.get_dummies(df[['date_received_dow']].astype(str))
doy_onehot = pd.get_dummies(df[['date_received_doy']].astype(str))
#date_onehot = pd.concat([month_onehot,dom_onehot,dow_onehot,doy_onehot],axis = 1)
#data_feature = pd.concat([data,date_onehot],axis = 1)
#data_feature = data_feature.drop(['date_received_month','date_received_dom','date_received_dow','date_received_doy'],axis = 1)

dfoff["month_onehot"]=month_onehot
dfoff["dom_onehot"]=dom_onehot
dfoff["dow_onehot"]=dow_onehot
dfoff["doy_onehot"]=doy_onehot

dfoff.loc[dfoff["month_onehot"].isnull(),"month_onehot"] = 0
dfoff.loc[dfoff["dom_onehot"].isnull(),"dom_onehot"] = 0
dfoff.loc[dfoff["dow_onehot"].isnull(),"dow_onehot"] = 0
dfoff.loc[dfoff["doy_onehot"].isnull(),"doy_onehot"] = 0

print(dfoff)

dftest["month_onehot"]=month_onehot.astype(int)
dftest["dom_onehot"]=dom_onehot.astype(int)
dftest["dow_onehot"]=dow_onehot.astype(int)
dftest["doy_onehot"]=doy_onehot.astype(int)


dftest.loc[dftest["month_onehot"].isnull(),"month_onehot"] = 0
dftest.loc[dftest["dom_onehot"].isnull(),"dom_onehot"] = 0
dftest.loc[dftest["dow_onehot"].isnull(),"dow_onehot"] = 0
dftest.loc[dftest["doy_onehot"].isnull(),"doy_onehot"] = 0

ValueError: Wrong number of items passed 4, placement implies 1

In [ ]:
date_received = dfoff['Date_received'].unique()
date_received = sorted( ~np.isnan(date_received))

date_received = sorted(dfoff[~np.isnan(dfoff['Date_received'])]['Date_received'])

print(date_received)
date_buy = dfoff['Date'].unique()
date_buy = sorted( ~np.isnan(date_buy))
print(date_buy)

date_buy = sorted(dfoff[~np.isnan(dfoff['Date'])]['Date'])
print('优惠券收到日期从',date_received[0],'到', date_received[-1])
print('消费日期从', date_buy[0], '到', date_buy[-1])

In [ ]:
couponbydate = dfoff[pd.notnull(dfoff['Date_received'])][['Date_received','Date']].groupby(['Date_received'], as_index=False).count()
couponbydate.columns = ['Date_received','count']
print(couponbydate)

buybydate = dfoff[pd.notnull(dfoff['Date']) &  pd.notnull(dfoff['Date_received'])] [['Date_received','Date']].groupby(['Date_received'], as_index=False).count()
buybydate.columns = ['Date_received','count']
print(buybydate)                                           

#print('無優惠券，購買商品條數',  dfoff[(pd.isnull(dfoff['Date_received'])) & (pd.notnull(dfoff['Date']))].shape[0])
#couponbydate = dfoff[dfoff['Date_received'] != 'null'][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
#couponbydate.columns = ['Date_received','count']
#buybydate = dfoff[(dfoff['Date'] != 'null') & (dfoff['Date_received'] != 'null')][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
#buybydate.columns = ['Date_received','count']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import date

sns.set_style('ticks')
sns.set_context("notebook", font_scale= 1.4)
plt.figure(figsize = (12,8))
date_received_dt = pd.to_datetime(date_received, format='%Y%m%d').unique()


plt.subplot(211)
plt.bar(date_received_dt, couponbydate['count'], label = 'number of coupon received' )
plt.bar(date_received_dt, buybydate['count'], label = 'number of coupon used')
#plt.yscale('log')
plt.ylabel('Count')
plt.legend()


plt.subplot(212)
plt.bar(date_received_dt, buybydate['count']/couponbydate['count'])
plt.ylabel('Ratio(coupon used/coupon received)')
plt.tight_layout()
plt.show()




In [ ]:
#Is_Holiday//優惠券領取日期是否屬於holiday

mask1=dfoff["Date_received"].isin(["20160101","20160102","20160103","20160207","20160208","20160209","20160210","20160211","20160212","20160213","20160402","20160403",
               "20160404","20160430","20160501","20160502","20160609","20160610","20160611"])
print(mask1)
dfoff["Is_Holiday"]=mask1.astype(int)
print(dfoff)

mask2=dftest["Date_received"].isin(["20160101","20160102","20160103","20160207","20160208","20160209","20160210","20160211","20160212","20160213","20160402","20160403",
               "20160404","20160430","20160501","20160502","20160609","20160610","20160611"])
dftest["Is_Holiday"]=mask2.astype(int)
print(dftest)

In [ ]:
#Is_Before_Holiday//優惠券領取日期是否屬於holiday

#employees[employees["Start Date"].between("2009-11-10","2010-01-20")]

mask3=dfoff["Date_received"].isin(["20160129","20160130","20160131","20160201","20160202","20160203","20160204","20160205","20160206","20160325","20160326","20160327",
               "20160328","20160329","20160330","20160428","20160429","20160331","20160401","20160601","20160602","20160603","20160604","20160605"])

print(mask3)
dfoff["Is_Before_Holiday"]=mask1.astype(int)
print(dfoff)

mask4=dftest["Date_received"].isin(["20160129","20160130","20160131","20160201","20160202","20160203","20160204","20160205","20160206","20160325","20160326","20160327",
               "20160328","20160329","20160330","20160428","20160429","20160331","20160401","20160601","20160602","20160603","20160604","20160605"])

dftest["Is_Before_Holiday"]=mask4.astype(int)
print(dftest)

In [ ]:
print('已有columns：',dfoff.columns.tolist())

In [ ]:

## Naive model
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train
    
df = dfoff[dfoff['label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)
train = df[df["is_train"]]
valid = df[~df["is_train"]]
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)
print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

In [ ]:
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'Is_Holiday', 'Is_Before_Holiday',
                    'month_onehot','dom_onehot','dow_onehot','doy_onehot',
                    'weekday_type'] + weekdaycols
print(len(original_feature),original_feature)

In [ ]:
predictors = original_feature
print(predictors)

def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

In [ ]:
model = check_model(train, predictors)

In [592]:
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]
valid1.head(2)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,discount_type,distance,month_onehot,dom_onehot,dow_onehot,doy_onehot,Is_Holiday,Is_Before_Holiday,is_train,pred_prob
0,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,1,...,1,0,4.0,29.0,4.0,120.0,0,0,False,0.024669
1,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,...,1,10,4.0,21.0,3.0,112.0,0,0,False,0.017321


In [593]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.741, Accuracy: 0.952


In [570]:
targetset = dftest.copy()
print(targetset.shape)
targetset = targetset[~targetset.Coupon_id.isnull()]
targetset.reset_index(drop=True, inplace=True)
testset = targetset[predictors].copy()

y_test_pred = model.predict_proba(testset[predictors])
test1 = testset.copy()
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)

(306313, 26)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [594]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)


In [595]:
### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("D:\\100days\data\ml100marathon-02-01\YuWen_7.csv", header=["uid", "label"], index=False) # submission format
out.head()

,uid,label
0,1000020_2705_20160519,0.116222
1,1000020_8192_20160513,0.090047
2,1000065_1455_20160527,0.070719
3,1000085_8067_20160513,0.072680
4,1000086_2418_20160613,0.062211
